Mayar Ariss (eddy.ariss19@imperial.ac.uk) (mayar313@mit.edu)

25 Mar 2024


Creation of ADAPTIC input files. 
Input data created in MATLAB. 

In [281]:
def generate_file(nod_names, xs, ys, zs, elm_names, grp_names, nod_name_lists, Rnod_names, restraints, init_nod_names, init_dir, init_type, init_value, mass_elem_names, mass_elem_idx, mass_elem_nod_names, vert_nodes,cpy_vec,vert_dis_1,vert_dis_2, gforcvec, sprd_vec, in_lod_els, in_lod_typ, in_lod_val1, in_lod_val2, in_lod_val3, output_file):
    content = """\
#
analysis 3d.full dynamics
#
default.parameters
mass.damping.parameter = 19.1853
stiffness.damping.parameter = 0.0018
#
materials
 mat.name    model       properties
#  {{{1
                  # En,   ft,   fc,   Gf1,   Gf3,  mu
     math     mcri  1900   0.04    6.2  0.01    20    0.2 &
                   # Es,   co,   tanf,   Gf2
                     16090    0.23    0.58    0.1
#
                   # En,   ft,   fc,   Gf1,   Gf3,  mu
     matv     mcri  1140   0.7    3.1   0.1    20   0.2 &
                   # Es,   co,   tanf,   Gf2
                     166900    1.5    0.8    0.1
#
                   # Ed,   co_d,   tanf_d,  Uult(mm) 
     matd     mcrd   580   0.17   0.42    10.5  &
                   # Eem, py, pr, miou, miour, Epm
                     4570  28560.8  5710.36  0.0  0.0  -264.4437 &
                    #TOM spring - IP
                    #kpl1 Fy   ftu  cF alpha beta flag_strength Fult(%)     w
                     0.1 0.8  0.3 0.4  0.8  0.01       1        1        0 &
                    #TOM spring - OOP
                    #Uult(*ufmax) cF_OOP alpha_OOP beta_OOP
                        10        0.4     0.8      0.01 
#
#rigid base
                  # En,   ft,   fc,   Gf1,   Gf3,  mu
     math_r     mcri  5e5  100    1000  10    200    0.2 &
                   # Es,   co,   tanf,   Gf2
                     5e5    10 1 10   
#
                   # En,   ft,   fc,   Gf1,   Gf3,  mu
     matv_r     mcri 5e5  100    1000  10    200 0.2 &
                   # Es,   co,   tanf,   Gf2
                     5e5    10 1 10 
#
                   # Ed,   co_d,   tanf_d,  Uult(mm)
     matd_r    mcrd   5e8   100   1    1000   &
                   # Eem, py, pr, miou, miour, Epm
                     4570  28560.8  5710.36  0.0  0.0  -264.4437 &
                    #TOM spring - IP
                    #kpl1 Fy   ftu  cF alpha beta flag_strength Fult(%)     w
                     0.1 0.8  0.3 0.4  0.8  0.01       1        1        0 &
                    #TOM spring - OOP
                    #Uult(*ufmax) cF_OOP alpha_OOP beta_OOP
                        10        0.4     0.8      0.01 

#  }}}1
#
groups
# {{{1
#
type = mcr4
  grp.name    mat.names    thickness gauss.points edge.flags density ratio.of.mass mass.damping
    macro1  math  matv math  matv matd   250    9  3  9  3   1 1 1 1   1.9e-9    0.5    0
    macro2  math_r  matv_r math_r  matv_r matd   250    9  3  9  3   1 1 1 1   1.9e-9    0.5    0
#
type = cpl2
grp.name  stiffness.parameters
cpy        rigid rigid 0 0 0 0

type = sprd
grp.name       direction    spread.nodes   ratios.of.spread
 g_forc_x x """

    # Add vertical displacement on top nodes data
    content += " ".join([f"{sprd_elem:<5}" for sprd_elem in sprd_vec])
    content += "\n"

    content += """\

type = cnm3
grp.name      mass
m1         2.548

#
#  }}}1
#
 structural.nodal
 nod.name           x            y            z
#   {{{1
"""

    # Add nodal coordinates
    for nod_name, x, y, z in zip(nod_names, xs, ys, zs):
        content += f"{nod_name:<10}{x:<12}{y:<12}{z:<12}\n"

    content += """\
#
1000 9000 2000 0
#   }}}1
#
 element.connectivity
 elm.name       grp.name     nod.name
#   {{{
#
"""

    # Add element connectivity
    for elm_name, grp_name, nod_name_list in zip(elm_names, grp_names, nod_name_lists):
        content += f"{elm_name:<10}{grp_name:<12}{' '.join(map(str, nod_name_list))}\n"

    content += """\
#
# impose same vertical displacement on top nodes
"""

    # Add vertical displacement on top nodes data
    for vert_nodes,cpy_vec,vert_dis_1,vert_dis_2 in zip(vert_nodes,cpy_vec,vert_dis_1,vert_dis_2):
        content += f"{vert_nodes:<5}{cpy_vec:<10}{vert_dis_1:<10}{vert_dis_2:<10}\n"

    content += """\
#   }}}1
#
1000 g_forc_x &
  1000 """

    # Add vertical displacement on top nodes data
    content += " ".join([f"{gforcvec_elem:<5}" for gforcvec_elem in gforcvec])
    content += "\n"

    content += """\
# Mass elements
"""

    # Add mass element data
    for mass_name, mass_idx, mass_nod in zip(mass_elem_names, mass_elem_idx, mass_elem_nod_names):
        content += f"{mass_name:<5}{mass_idx:<10}{mass_nod:<10}\n"

    content += """\
  

 restraints
 nod.name      direction
 #   {{{1
"""

    # Add nodal coordinates restraints
    for rn, r in zip(Rnod_names, restraints):
        content += f"{rn:<10}{r:<10}\n"

    content += """\
    
    1000 y+z+rx+ry+rz
#   }}}1
#
linear.curves
#   {{{1
 start.time = 0
 crv.name = c1
 file = Short_panel.acc.dat    
 first.line = 2
 #last.line = 1200
 #format = (f8.6,3x,f9.6)
#   }}}1
#
 applied.loading
 dynamic.loads
 nod.name       direction        type        crv.name          value 
#   {{{1
1000    x            a    c1   1.0
#   }}}1
 initial.loads
elm.name  type value
"""

    # Add initial.loads element data
    for ile, ilt, ilv1, ilv2, ilv3 in zip(in_lod_els, in_lod_typ, in_lod_val1, in_lod_val2, in_lod_val3):
        content += f"{ile:<5}   {ilt:<10}{ilv1:<10}{ilv2:<10}{ilv3:<10}\n"

    content += """\

nod.name       direction        type          value 
#   {{{1
"""

    # Add nodal initial loads
    for nod_name, x, y, z in zip(init_nod_names, init_dir, init_type, init_value):
        content += f"{nod_name:<10}{x:<10}{y:<10}{z:<12}\n"

    content += """\
#   }}}1
1000    x           v  -0.000615451

#
 equilibrium.stages
   end.of.stage    steps
    4000    4000
#
 solution
  #solver = mumps
#
#dynamic analysis parameters
 integration.scheme
 scheme = hilber.hughes.taylor
  alpha = -0.33
#
 iterative.strategy
   number = 60
   initial.reformations = 60
   step.reduction = 60
   divergence.iteration = 60
   maximum.convergence =  1e7
#
 convergence.criteria
   tolerance = 1.000000e-05
   work.ref = 1.000000e+06
#
 output
 frequency 10 stress
#
end
"""
    # Write content to the output file
    with open(output_file, "w") as file:
        file.write(content)


In [282]:
# Example usage
import numpy as np

# Building name
name='BUILDING2_corr'
folder='building_2_disc'

for i in 1, 2:

    for let in 'X', 'Y':

        # Load data from sorted_coordinates.txt
        data = np.genfromtxt(f"{folder}/W{i}_{let}_{name}_sorted_coordinates.txt", skip_header=1, dtype=str)
        nod_names = data[:, 0]
        xs = data[:, 1].astype(float) #x:1 y:2
        ys = data[:, 2].astype(float) #x:3 y:3
        zs = data[:, 3].astype(float) #x:2 y:1

        elm_names = []
        grp_names = []
        nod_name_lists = []

        with open(f"{folder}/W{i}_{let}_{name}_elem_coordinates.txt", "r") as file:
            next(file)  # Skip the header
            for line in file:
                parts = line.strip().split("\t")
                elm_names.append(parts[0])
                grp_names.append(parts[1])
                nod_name_lists.append(parts[2].split())

        # Define restraints data
        # Load data from restraints.txt
        rest_data = np.genfromtxt(f"{folder}/W{i}_{let}_{name}_restraints.txt", skip_header=1, dtype=str)
        Rnod_names = rest_data[:, 0]
        restraints = rest_data[:, 1]

        # Define initial data
        # Load data from initial_loads.txt
        init_data = np.genfromtxt(f"{folder}/W{i}_{let}_{name}_initial_loads.txt", skip_header=1, dtype=str)
        init_nod_names = init_data[:, 0]
        init_dir = init_data[:, 1]
        init_type = init_data[:, 2]
        init_value = init_data[:, 3]

        # Define mass elems data
        # Load data from mass_elems.txt
        mass_data = np.genfromtxt(f"{folder}/W{i}_{let}_{name}_mass_elems.txt", skip_header=1, dtype=str)
        mass_elem_names = mass_data[:, 0]
        mass_elem_idx = mass_data[:, 1]
        mass_elem_nod_names = mass_data[:, 2]

        # Define vertcal imposed displacement data on top nodes
        # Load data from vert_disp.txt
        vert_disp = np.genfromtxt(f"{folder}/W{i}_{let}_{name}_vert_disp.txt", skip_header=1, dtype=str)
        vert_nodes = vert_disp[:, 0]
        cpy_vec = vert_disp[:, 1]
        vert_dis_1 = vert_disp[:, 2]
        vert_dis_2 = vert_disp[:, 3]

        # Define g_forc_x nodes
        # Load data from gforcvec.txt
        gforcvec_data = np.genfromtxt(f"{folder}/W{i}_{let}_{name}_gforcvec.txt", skip_header=1, dtype=str)
        gforcvec = gforcvec_data

        # Define sprd nodes
        # Load data from sprd.txt
        sprd_data = np.genfromtxt(f"{folder}/W{i}_{let}_{name}_sprd.txt", skip_header=1, dtype=str)
        sprd_vec = sprd_data

        # Define elem in lod data
        # Load data from in_lod.txt
        in_lod_data = np.genfromtxt(f"{folder}/W{i}_{let}_{name}_in_lod.txt", skip_header=1, dtype=str)
        in_lod_els = in_lod_data[:, 0]
        in_lod_typ = in_lod_data[:, 1]
        in_lod_val1 = in_lod_data[:, 2]
        in_lod_val2 = in_lod_data[:, 3]
        in_lod_val3 = in_lod_data[:, 4]

        generate_file(nod_names, xs, ys, zs, elm_names, grp_names, nod_name_lists, Rnod_names, restraints, init_nod_names, init_dir, init_type, init_value, mass_elem_names, mass_elem_idx, mass_elem_nod_names,vert_nodes,cpy_vec,vert_dis_1,vert_dis_2,gforcvec, sprd_vec, in_lod_els, in_lod_typ, in_lod_val1, in_lod_val2, in_lod_val3, f"{folder}/generated_file_disc_W{i}_{let}_{name}.txt")

In [283]:
# Building name
name='BUILDING2_corr'
folder='building_2_disc'

for i in 1, 2:

    for let in 'X', 'Y':

        # Open the .txt file for reading
        with open(f"{folder}/generated_file_disc_W{i}_{let}_{name}.txt", "r") as txt_file:
            # Open a new .dat file for writing
            with open(f"{folder}/generated_file_disc_W{i}_{let}_{name}.dat", "w") as dat_file:
                # Read the content of the .txt file
                content = txt_file.read()
                # Write the content to the .dat file
                dat_file.write(content)